In [1]:
import pandas as pd
import numpy 
import csv
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format

In [2]:

# df = pd.read_csv('all_comments_since_2015.csv', error_bad_lines=False, warn_bad_lines=True)
# df = pd.read_csv('all_comments_since_2015.csv', delimiter=',', quotechar='"', error_bad_lines=False)
# df = pd.read_csv('all_comments_since_2015.csv', on_bad_lines='skip')
# df = pd.read_csv('all_comments_since_2015.csv', engine='python')
# df = pd.read_csv('all_comments_since_2015.csv', quoting=csv.QUOTE_NONE)


In [3]:
def handle_bad_line(bad_line):
    print(f"Bad line: {bad_line}")
    return None  # Returning None will skip the bad line
c_df = pd.read_csv('all_comments_since_2015.csv', on_bad_lines='skip', engine='python')
a_df = pd.read_csv('./author_profiles.csv')

print(c_df.shape, c_df.groupby('author').nunique().shape)
print(a_df.shape)


(1541609, 17) (8388, 16)
(10295, 38)


In [4]:
big_5_a_df = a_df.dropna(subset=['agreeableness', 'openness', 'conscientiousness', 'extraversion','neuroticism'])
# big_5_a_df.head(2)[['author', 'mbti', 'type', 'agreeableness', 'openness', 'conscientiousness', 'extraversion', 'neuroticism',  'num_comments', 'en_comments', 'is_female', 'is_female_pred', 'is_female_proba']]
# big_5_a_df.groupby('mbti').size()
all_df = pd.merge(c_df, big_5_a_df, on='author', how='inner') 
# all_df = all_df[all_df['lang'] =='en']
print(big_5_a_df.shape, all_df.shape)

(1568, 38) (58252, 54)


In [5]:
print(f"total author = {a_df.shape}")
print(f"total unique author = {a_df.groupby('author').nunique().shape}")
print(f"total unique big 5 author = {big_5_a_df.groupby('author').nunique().shape}")
print(30*"-")
print(f"total comments = {c_df.shape}")
print(f"total unique author in comments = {c_df.groupby('author').nunique().shape}")
print(f"total big 5 author w/ comments = {all_df.groupby('author').nunique().shape}")
print(f"total big 5 authers' comments {all_df.shape}")

total author = (10295, 38)
total unique author = (10295, 37)
total unique big 5 author = (1568, 37)
------------------------------
total comments = (1541609, 17)


total unique author in comments = (8388, 16)
total big 5 author w/ comments = (222, 53)
total big 5 authers' comments (58252, 54)


In [8]:
df = all_df[all_df['lang'] =='en']
df = df[df['word_count_quoteless']>1]
s = df[['author', 'body', 'agreeableness', 'openness', 'conscientiousness', 'extraversion', 'neuroticism']]
s= s.rename(columns= {'author':'#AUTHID', 'body': 'STATUS'})
s.shape

(51818, 7)

In [9]:
s.describe()

,agreeableness,openness,conscientiousness,extraversion,neuroticism
count,51818.0,51818.0,51818.0,51818.0,51818.0
mean,35.4,68.7,33.6,35.7,43.9
std,29.1,22.5,31.3,32.1,30.2
min,0.0,4.0,0.0,0.0,0.0
25%,9.0,59.0,10.0,4.0,15.0
50%,26.0,73.0,16.0,24.0,41.0
75%,57.0,86.0,55.0,70.0,72.0
max,99.0,99.0,98.0,99.0,99.0


In [7]:
cols = ['openness', 'extraversion', 'neuroticism', 'agreeableness','conscientiousness']
n_cols = ['cOPN', 'cEXT', 'cNEU', 'cAGR', 'cCON']
norm_cols = ['nOPN', ' EXT', 'nNEU', 'nAGR', 'nCON']
std_cols = ['sOPN', 'sEXT', 'sNEU', 'sAGR', 'sCON']

from sklearn.preprocessing import MinMaxScaler, StandardScaler
min_max_scaler = MinMaxScaler()
standard_scaler = StandardScaler()

for col, n_col, norm_col, std_col in zip(cols, n_cols, norm_cols, std_cols):
    s[[n_col]] = s[[col]]/20
    s[[norm_col]] = min_max_scaler.fit_transform(s[[col]])
    s[[std_col]] = standard_scaler.fit_transform(s[[col]])


In [87]:
means = {}
for n_col, norm_col, std_col in zip( n_cols, norm_cols, std_cols):
    means[n_col] = s[n_col].mean()
    means[norm_col] = s[norm_col].mean()
    means[std_col] = s[std_col].mean()
    s[n_col] = s[n_col] > means[n_col]
    s[norm_col] = s[n_col] > means[norm_col]
    s[std_col] = s[std_col] > means[std_col]
s[n_cols] = s[n_cols].replace({True: 1, False: 0})  
s[norm_cols] = s[norm_cols].replace({True: 1, False: 0})  
s[std_cols] = s[std_cols].replace({True: 1, False: 0})  

s.head(2)

,#AUTHID,STATUS,agreeableness,openness,conscientiousness,extraversion,neuroticism,cOPN,nOPN,sOPN,...,sEXT,cNEU,nNEU,sNEU,cAGR,nAGR,sAGR,cCON,nCON,sCON
0,MetricExpansion,Those stats come from the test. [Echoing the c...,30.0,70.0,15.0,15.0,50.0,1,1,1,...,0,1,1,1,0,0,0,0,0,0
1,MetricExpansion,"That's great to hear! I hope you know that, de...",30.0,70.0,15.0,15.0,50.0,1,1,1,...,0,1,1,1,0,0,0,0,0,0


In [134]:
for col, n_col, norm_col, std_col in zip(cols, n_cols, norm_cols, std_cols):
    print(s[[col] + [n_col]  + [norm_col] + [std_col]][s[n_col] != s[std_col]])
# s[['cAGR', 'nAGR', 'sAGR', 'agreeableness']]

Empty DataFrame
Columns: [openness, cOPN, nOPN, sOPN]
Index: []
Empty DataFrame
Columns: [extraversion, cEXT,  EXT, sEXT]
Index: []
Empty DataFrame
Columns: [neuroticism, cNEU, nNEU, sNEU]
Index: []
Empty DataFrame
Columns: [agreeableness, cAGR, nAGR, sAGR]
Index: []
Empty DataFrame
Columns: [conscientiousness, cCON, nCON, sCON]
Index: []


In [114]:
# s.to_csv('../pandora_to_big5.csv')

In [152]:
s_cols = ['sOPN', 'sEXT', 'sNEU', 'sAGR', 'sCON']
n_cols = ['cOPN', 'cEXT', 'cNEU', 'cAGR', 'cCON']
norm_cols = ['nOPN', ' EXT', 'nNEU', 'nAGR', 'nCON']
std_cols = ['sdOPN', 'sdEXT', 'sdNEU', 'sdAGR', 'sdCON']

s = pd.read_csv('../mypersonality.csv', encoding='Windows-1252')
s= s[['#AUTHID', 'STATUS'] + s_cols + n_cols]

from sklearn.preprocessing import MinMaxScaler, StandardScaler
min_max_scaler = MinMaxScaler()
standard_scaler = StandardScaler()

for col, norm_col, std_col in zip(s_cols, norm_cols, std_cols):
    s[[norm_col]] = min_max_scaler.fit_transform(s[[col]])
    s[[std_col]] = standard_scaler.fit_transform(s[[col]])
s.head(1)

,#AUTHID,STATUS,sOPN,sEXT,sNEU,sAGR,sCON,cOPN,cEXT,cNEU,...,nOPN,sdOPN,EXT,sdEXT,nNEU,sdNEU,nAGR,sdAGR,nCON,sdCON
0,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,4.4,2.6,3.0,3.1,3.2,y,n,y,...,0.8,0.5,0.4,-0.8,0.5,0.5,0.4,-0.7,0.5,-0.3


In [153]:
means = {}
for s_col, norm_col, std_col in zip(s_cols, norm_cols, std_cols):
    means[s_col] = s[s_col].mean()
    means[norm_col] = s[norm_col].mean()
    means[std_col] = s[std_col].mean()
    s[s_col] = s[s_col] > means[s_col]
    s[norm_col] = s[norm_col] > means[norm_col]
    s[std_col] = s[std_col] > means[std_col]
s[s_cols] = s[s_cols].replace({True: 1, False: 0})  
s[norm_cols] = s[norm_cols].replace({True: 1, False: 0})  
s[std_cols] = s[std_cols].replace({True: 1, False: 0})  
# s[n_cols] = s[std_cols].replace({'y': 1, 'n': 0})  

In [155]:
for col, n_col, norm_col, std_col in zip(s_cols, n_cols, norm_cols, std_cols):
    print(s[[col] + [n_col]  + [norm_col] + [std_col]][s[std_col] != s[norm_col]])

Empty DataFrame
Columns: [sOPN, cOPN, nOPN, sdOPN]
Index: []
Empty DataFrame
Columns: [sEXT, cEXT,  EXT, sdEXT]
Index: []
Empty DataFrame
Columns: [sNEU, cNEU, nNEU, sdNEU]
Index: []
Empty DataFrame
Columns: [sAGR, cAGR, nAGR, sdAGR]
Index: []
Empty DataFrame
Columns: [sCON, cCON, nCON, sdCON]
Index: []


In [178]:
s[['cAGR', 'nAGR', 'sdAGR', 'sAGR']].sample(5)

,cAGR,nAGR,sdAGR,sAGR
6149,y,1,1,1
3116,y,1,1,1
2939,y,0,0,0
8113,n,0,0,0
5039,n,0,0,0


In [183]:
s[['STATUS'] +['cAGR', 'nAGR', 'sdAGR', 'sAGR']].iloc[2939]

STATUS    wants to be rich!!!!!!!!!!!!!! filthy. c'mon l...
cAGR                                                      y
nAGR                                                      0
sdAGR                                                     0
sAGR                                                      0
Name: 2939, dtype: object